In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [130]:
activity_dir = 'C:/Users/SAMSUNG/Desktop/new/빅콘테스트/2019빅콘테스트_챔피언스리그_데이터_수정/train_activity.csv'
combat_dir = 'C:/Users/SAMSUNG/Desktop/new/빅콘테스트/2019빅콘테스트_챔피언스리그_데이터_수정/train_combat.csv'
trade_dir = 'C:/Users/SAMSUNG/Desktop/new/빅콘테스트/2019빅콘테스트_챔피언스리그_데이터_수정/train_trade.csv'
label_dir ='C:/Users/SAMSUNG/Desktop/new/빅콘테스트/2019빅콘테스트_챔피언스리그_데이터_수정/train_label_add.csv'
pay_dir = 'C:/Users/SAMSUNG/Desktop/new/빅콘테스트/2019빅콘테스트_챔피언스리그_데이터_수정/train_payment.csv'

train_activity = pd.read_csv(activity_dir)
train_combat = pd.read_csv(combat_dir)
train_trade = pd.read_csv(trade_dir)
train_label = pd.read_csv(label_dir)
del train_label['Unnamed: 0']
train_payment = pd.read_csv(pay_dir)

# ---------------------activity + combat------------------------

In [132]:
# bs 서버를 제외하고 수행
train_activity = train_activity[train_activity.server != 'bs']
train_activity = train_activity[train_activity.playtime != 0]

### 변수 추가 시작

In [133]:
# 3. 전체  - 주캐  (나누는 건 더 나쁨) 안 빼고 그냥 주캐 시간만 추가하는 게 제일 나음
# 전반적으로 주캐 값만 쓰는 게 더 낫지만 엎치락 뒤치락 하는 경우도 있음

# 1) activity에 combat에 없는 서버 활동 지우고 합치기
com_act = pd.merge(train_combat, train_activity, on=['acc_id','char_id','day'], how='outer')
com_act_40000 = com_act.groupby('acc_id').sum().reset_index()

# 2) 주캐와 부캐로 나눠서 sum (부캐가 없으면? =1)
com_act2 = com_act.groupby('acc_id')['level'].max().reset_index()
column = com_act[['acc_id','level','pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count']]
# day, char_id, server_x, server_y, class 제거한 상태
com_act3 = pd.merge(com_act2,column, on=['acc_id','level'], how ='inner')
com_act3 = com_act3.groupby('acc_id').sum().reset_index()
act_40000_max = pd.merge(com_act_40000, com_act3, on=['acc_id'])

class 제거

### 주케 특성 vs 전체 특성 비교

In [134]:
# 주캐
act_40000_best = act_40000_max[['acc_id', 'day', 'char_id', 'class', 'temp_cnt_x', 'private_shop_x', 
                               'level_x', 'party_exp_x', 'pledge_cnt_y', 'random_attacker_cnt_y', 'random_defender_cnt_y',
       'same_pledge_cnt_y', 'etc_cnt_y', 'num_opponent_y',
       'playtime_y', 'npc_kill_y', 'solo_exp_y', 'quest_exp_y',
       'rich_monster_y', 'death_y', 'revive_y', 'exp_recovery_y', 'fishing_y',
       'game_money_change_y', 'enchant_count_y']]

act_40000_best.columns = ['acc_id', 'day', 'char_id', 'class', 'temp_cnt', 'private_shop', 
                               'level', 'party_exp', 'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent',
       'playtime', 'npc_kill', 'solo_exp', 'quest_exp',
       'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'game_money_change', 'enchant_count']

In [135]:
# 7. 생존과 부활 비교
act_40000_best['forgive'] = act_40000_best['death'] - act_40000_best['revive']

C:\Users\SAMSUNG\Desktop\new\anacanda\envs\mypython\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [136]:
com_act = pd.merge(train_combat, train_activity, on=['acc_id','char_id','day'],
                   how='inner').drop(['server_x','server_y'],axis=1)

# 변수별 횟수 세는 변수 추가
feature_count = com_act.groupby(['acc_id','day']).sum().reset_index()
for col in ['pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'playtime',
       'npc_kill', 'solo_exp', 'party_exp', 'quest_exp', 'rich_monster',
       'death', 'revive', 'exp_recovery', 'fishing', 'private_shop',
       'game_money_change', 'enchant_count']:
    col_count = col + '_count'
    feature_count[col_count] = 0
for col in ['pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'playtime',
       'npc_kill', 'solo_exp', 'party_exp', 'quest_exp', 'rich_monster',
       'death', 'revive', 'exp_recovery', 'fishing', 'private_shop',
       'game_money_change', 'enchant_count']:
    col_count = col + '_count'
    feature_count[col_count] = np.where(feature_count[col]!=0,1,0)
    
feature_count = feature_count.groupby('acc_id').sum().reset_index().drop(['day', 'char_id', 'class', 'level', 'pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change', 'enchant_count'], axis=1)

In [137]:
act_40000_best = pd.merge(act_40000_best, feature_count, on ='acc_id')

In [138]:
# 9. 접속 중단 횟수

# 실제 접속일  데이터
real_day = train_activity[['acc_id','day','playtime']]
real_day = real_day.groupby(['acc_id','day']).sum().reset_index()
# 전체 일자 데이터 (40000*28)
day = np.zeros(shape=(20000*28,2))

# acc_id 40000개 추출
acc_id = train_activity[['acc_id','playtime']]
acc_id = acc_id.groupby('acc_id').sum().reset_index()
acc_id = acc_id['acc_id']
acc_id = acc_id.values

# day에 acc_id와 1~28일 입력 후 데이터프레임으로 변환
for i in range(0,20000):
    for j in range(0,28):
        n = 28*i+j
        day[n][0] = acc_id[i]
        day[n][1] = j+1
day_df = pd.DataFrame(day, columns=['acc_id','day'])

# 만든 데이터프레임에 실제 접속일 데이터 merge
stop_count_df = pd.merge(day_df, real_day,  on=['acc_id','day'],how='outer')
# 접속하지 않은 날은 0, 접속일은 1로 표시
stop_count_df['login'] = np.where(stop_count_df['playtime'].isnull(), 0, 1)
stop_count_df['stop'] = 0
#stop_count_df['day'] = np.where(stop_count_df['playtime'].isnull(), 0, stop_count_df['day'])
# 이제 필요 없는 playtime 제거

stop_count_df = stop_count_df.drop('playtime',axis=1)

playtime 변수 제거

In [139]:
stop_count_np = stop_count_df.values

for i in range(0,len(stop_count_df)-1):
    if stop_count_np[i+1][2] - stop_count_np[i][2] == 1:
        stop_count_np[i+1][3] = 1
#for i in range(0,40000):
#    if stop_count_np[i*28][2] == 0:
#        stop_count_np[i*28][3] = -1
stop_count_df = pd.DataFrame(stop_count_np, columns = ['acc_id', 'day', 'login', 'stop'])

In [140]:
stop_count_40000 = stop_count_df.groupby('acc_id').sum().reset_index()

### 군집화 변수 추가

In [141]:
# 접속패턴 군집 추가
# 실제 접속일  데이터
real_day = train_activity[['acc_id','day','playtime']]
real_day = real_day.groupby(['acc_id','day']).sum().reset_index()

# 전체 일자 데이터 (40000*28)
day = np.zeros(shape=(40000*28,2))

# acc_id 40000개 추출
acc_id = train_activity[['acc_id','playtime']]
acc_id = acc_id.groupby('acc_id').sum().reset_index()
acc_id = acc_id['acc_id']
acc_id = acc_id.values

# day에 acc_id와 1~28일 입력 후 데이터프레임으로 변환
for i in range(0,40000):
    for j in range(0,28):
        n = 28*i+j
        day[n][0] = acc_id[i]
        day[n][1] = j+1
day_df = pd.DataFrame(day, columns=['acc_id','day'])

# 만든 데이터프레임에 실제 접속일 데이터 merge
stop_count_df = pd.merge(day_df, real_day,  on=['acc_id','day'],how='outer')
# 접속하지 않은 날은 0, 접속일은 1로 표시
stop_count_df['login'] = np.where(stop_count_df['playtime'].isnull(), 0, 1)
stop_count_df['stop'] = 0


stop_count_df = stop_count_df.drop('playtime',axis=1)

stop_count_np = stop_count_df.values

for i in range(0,len(stop_count_df)-1):
    if stop_count_np[i+1][2] - stop_count_np[i][2] == 1:
        stop_count_np[i+1][3] = 1

stop_count_df = pd.DataFrame(stop_count_np, columns = ['acc_id', 'day', 'login', 'stop'])


# 접속패턴 데이터셋을 만드는 데 필요한 acc_id와 login 컬럼만 stop_count_df에서 뽑아옵니다. 
day_pattern_df = stop_count_df[['acc_id','login']]

# 편한 인덱싱을 위해 데이터프레임을 넘파이 배열로 바꿔줍니다.
day_pattern_np = day_pattern_df.values

# 열은 'acc_id + 28일' 이므로 29입니다.
pattern_np = np.zeros(shape=(40000,29))
for i in range(0,40000):
    for j in range(0,28):
        # 28번씩 반복된 day_pattern_np의 acc_id를 28번에 한 번씩 갱신해서 pattern_np에 입력합니다.
        pattern_np[i][0] = day_pattern_np[i*28][0]
        # 각 acc_id마다 행으로 28 단위로 입력된 login을 열 방향으로 입력합니다. 
        # 0열은 acc_id를 입력하므로 j+1을 해줍니다.
        pattern_np[i][j+1] = day_pattern_np[28*i+j][1]
        
# columns 리스트 생성 
col = ['acc_id']
col.extend(list(range(1,29,1)))
# columns 지정
day_pattern_df = pd.DataFrame(pattern_np, columns = col)
# 접속패턴 군집 변수 생성
login = day_pattern_df.drop(['acc_id'],axis=1)

model = KMeans(n_clusters=2,algorithm='auto')
model.fit(login)
predict = pd.DataFrame(model.predict(login))
predict.columns=['login_clt']
r_login = pd.concat([login,predict],axis=1)
login_cluster = pd.concat([day_pattern_df['acc_id'],r_login],axis=1)

In [142]:
# 전투패턴 군집
com = train_combat.groupby(['acc_id']).sum().reset_index()
com_pattern = com[['pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt']]

# create model and prediction
model = KMeans(n_clusters=3,algorithm='auto') # 3이 좋을지, 2가 좋을지는 경험적으로 판단해야 함.

model.fit(com_pattern)

predict = pd.DataFrame(model.predict(com_pattern))

predict.columns=['combat_clt']
r_combat = pd.concat([com_pattern,predict],axis=1)
combat_cluster = pd.concat([day_pattern_df['acc_id'],r_combat['combat_clt']],axis=1)

In [143]:
# 플레이 패턴 군집 -> playtime 을 이용한 군집화
# 실제 접속일  데이터
real_day = train_activity[['acc_id','day','playtime']]
real_day = real_day.groupby(['acc_id','day']).sum().reset_index()

# 전체 일자 데이터 (40000*28)
day = np.zeros(shape=(40000*28,2))

# acc_id 40000개 추출
acc_id = train_activity[['acc_id','playtime']]
acc_id = acc_id.groupby('acc_id').sum().reset_index()
acc_id = acc_id['acc_id']
acc_id = acc_id.values

# day에 acc_id와 1~28일 입력 후 데이터프레임으로 변환
for i in range(0,40000):
    for j in range(0,28):
        n = 28*i+j
        day[n][0] = acc_id[i]
        day[n][1] = j+1
day_df = pd.DataFrame(day, columns=['acc_id','day'])

# 만든 데이터프레임에 실제 접속일 데이터 merge
playtime_df = pd.merge(day_df, real_day,  on=['acc_id','day'],how='outer')

play_pattern_df = playtime_df[['acc_id','playtime']]
# 편한 인덱싱을 위해 데이터프레임을 넘파이 배열로 바꿔줍니다.
play_pattern_np = play_pattern_df.values

# 열은 'acc_id + 28일' 이므로 29입니다.
p_pattern_np = np.zeros(shape=(40000,29))
for i in range(0,40000):
    for j in range(0,28):
        # 28번씩 반복된 day_pattern_np의 acc_id를 28번에 한 번씩 갱신해서 pattern_np에 입력합니다.
        p_pattern_np[i][0] = play_pattern_np[i*28][0]
        # 각 acc_id마다 행으로 28 단위로 입력된 login을 열 방향으로 입력합니다. 
        # 0열은 acc_id를 입력하므로 j+1을 해줍니다.
        p_pattern_np[i][j+1] = play_pattern_np[28*i+j][1]
        
col = ['acc_id']
col.extend(list(range(1,29,1)))
play_pattern_df = pd.DataFrame(p_pattern_np, columns = col)

# 완성입니다. 
play_pattern_df = play_pattern_df.fillna(0)

play_pattern = play_pattern_df.drop('acc_id',axis=1)

# create model and prediction
model = KMeans(n_clusters=3,algorithm='auto')

model.fit(play_pattern)

predict = pd.DataFrame(model.predict(play_pattern))

predict.columns=['playtime_clt']

r_play = pd.concat([play_pattern,predict],axis=1)
playtime_cluster = pd.concat([play_pattern_df['acc_id'],r_play[['playtime_clt']]],axis=1)

In [144]:
# 사냥 패턴 군집
# 유저별 exp 컬럼만 추출
exp = train_activity[['day', 'acc_id', 'char_id', 'solo_exp', 'party_exp', 'quest_exp']]
exp = exp.groupby('acc_id').sum().reset_index().drop(['day', 'char_id'],axis=1)
exp_pattern = exp.drop('acc_id',axis=1)

# create model and prediction
model = KMeans(n_clusters=3,algorithm='auto')
model.fit(exp_pattern)
predict = pd.DataFrame(model.predict(exp_pattern))
predict.columns=['exp_clt']

r_exp = pd.concat([exp_pattern,predict],axis=1)
exp_cluster = pd.concat([exp['acc_id'],r_exp[['exp_clt']]],axis=1)

In [145]:
act_40000_best = pd.merge(act_40000_best,login_cluster, on='acc_id')
act_40000_best = pd.merge(act_40000_best,combat_cluster, on='acc_id')
act_40000_best = pd.merge(act_40000_best,playtime_cluster, on='acc_id')
act_40000_best = pd.merge(act_40000_best,exp_cluster, on='acc_id')

### 여기서부터는 주캐가 아닌 전체 캐릭터를 대상으로 변수 생성

In [146]:
# 40000명으로 압축한 데이터셋
data3 = train_combat.groupby('acc_id').sum().drop(['char_id','level','class','day'],axis=1).reset_index()
data2 = train_activity.groupby('acc_id').sum().drop(['char_id','day'],axis=1).reset_index()
data1 = pd.merge(data2, data3, on ='acc_id')

### combat data 함수화

In [147]:
# 모든 변수는 40000행 데이터셋에 맞춰서 merge 됩니다. 
def std(x): return np.std(x)


def num_combat(train_combat, data):
    num_combat = train_combat[['acc_id','pledge_cnt','random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt']]
    # 전투기록이 있으면 1, 아니면 0
    num_combat['pledge_cnt'] = np.where(train_combat['pledge_cnt']>0, 1,0)
    num_combat['random_attacker_cnt'] = np.where(train_combat['random_attacker_cnt']>0, 1,0)
    num_combat['random_defender_cnt'] = np.where(train_combat['random_defender_cnt']>0, 1,0)
    num_combat['temp_cnt'] = np.where(train_combat['temp_cnt']>0, 1,0)
    num_combat['same_pledge_cnt'] = np.where(train_combat['same_pledge_cnt']>0, 1,0)
    num_combat['etc_cnt'] = np.where(train_combat['etc_cnt']>0, 1,0)

    # 합산 및 컬럼명 변경
    num_combat = num_combat.groupby('acc_id').sum().reset_index()

    num_combat.rename(columns={'pledge_cnt':'day_pledge', 'random_attacker_cnt':'day_attack', 'random_defender_cnt':'day_defend', 
                           'temp_cnt':'day_temp', 'same_pledge_cnt':'day_same', 'etc_cnt':'day_etc'},inplace=True)
    data2 = pd.merge(data, num_combat, on = 'acc_id')
    return data2

# 6. 레벨이 올랐는가?
# 의문점: 고렙 유저와 저렙 유저에 차이가 있어서 가중치를 줘야하나?
def deff_level(train_combat,data):    
    # 관련 컬럼 불러오기
    level =  train_combat[['acc_id','char_id','day','level']]
    # 캐릭터별 max 레벨과 min 레벨 생성
    level_max = level.groupby(['acc_id','char_id']).max().reset_index()
    level_min = level.groupby(['acc_id','char_id']).min().reset_index()
    # 레벨 데이터를  acc_id,char_id 단위 그룹으로 재정의(day 압축)
    level = level.groupby(['acc_id','char_id']).sum().reset_index()
    # max, min 컬럼 추가
    level['max'] = level_max['level']
    level['min'] = level_min['level']
    # 차이 계산
    level['deff_level'] = level['max'] - level['min']
    # acc_id로 그룹화 후, 필요 없는 컬럼 제거
    level = level.groupby(['acc_id']).sum().reset_index().drop(['max','min','day','level','char_id'],axis=1)
    # deff_level 추가
    data2 = pd.merge(data, level, on = 'acc_id')
    return data2

# 5. 캐릭터별 각 전투 변수의 28일 간 표준편차 (접속한 날만 대상으로 표준편차 구함)
def std_combat(train_combat,data):
    std_combat = train_combat[['acc_id','day','char_id','pledge_cnt','random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt']]
    # 케릭터 별로 28일의 표준편차 구하고, day 없애고, 모든 값이 0이고 1행인 애들이 NaN이 나옴
    # 1안: 접속 안한 날은 전투시간 =0인 날로 간주하고 n=28로 분산 계산한다
    # 2안: 접속일수를 분모로 분산을 계산한다.
    # 3안: 유저 단위로 합쳐서 1,2안 중 하나를 계산한다
    # 4안: 다 해본다.
    
    std_combat = std_combat.groupby(['acc_id','char_id']).agg(std).reset_index().drop('day',axis=1).fillna(0)
    std_combat.rename(columns={'pledge_cnt':'std_pledge', 'random_attacker_cnt':'std_attack', 'random_defender_cnt':'std_defend', 
                           'temp_cnt':'std_temp', 'same_pledge_cnt':'std_same', 'etc_cnt':'std_etc'},inplace=True)
    
    # 일단 캐릭터별 표준편차를 유저 단위로 합함
    std_combat = std_combat.groupby('acc_id').sum().reset_index().drop('char_id',axis=1)
    data2 = pd.merge(data, std_combat, on = ['acc_id'])
    return data2

# 유저별 각 전투 변수의 28일 간 표준편차(접속한 날만 대상으로 표준편차 구함)
def acc_std_combat(train_combat,data):
    acc_std_combat = train_combat[['acc_id','day','char_id','pledge_cnt','random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt']]
    
    acc_std_combat = acc_std_combat.groupby(['acc_id']).agg(std).reset_index().drop('day',axis=1).fillna(0)
    acc_std_combat.rename(columns={'pledge_cnt':'acc_std_pledge', 'random_attacker_cnt':'acc_std_attack', 'random_defender_cnt':'acc_std_defend', 
                           'temp_cnt':'acc_std_temp', 'same_pledge_cnt':'acc_std_same', 'etc_cnt':'acc_std_etc'},inplace=True)
    
    acc_std_combat = acc_std_combat.drop('char_id',axis=1)
    data2 = pd.merge(data, acc_std_combat, on = ['acc_id'])
    return data2


# 총 사용 캐릭터 개수(char_count), 총 접속일수(day_count)     
def char_day_count(train_combat,data):    
    char_count = train_combat.groupby(['day','acc_id'])['char_id'].count()
    char_count = char_count.reset_index()
    char_count = char_count.groupby('acc_id').count()
    char_count = char_count.reset_index()
    char_count.rename(columns={'day':'day_count','char_id':'char_count'},inplace=True)
    data2 = pd.merge(data, char_count, on = ['acc_id'])
    return data2

# 보유한 캐릭터 개수
def char_max(train_combat,data):
    char_max = train_combat.groupby(['day','acc_id'])['char_id'].count()
    char_max = char_max.reset_index()
    char_max = char_max.groupby('acc_id').max()
    char_max = char_max.reset_index().drop('day',axis=1)
    char_max.rename(columns={'char_id':'char_max'},inplace=True)
    data2 = pd.merge(data, char_max, on = ['acc_id'])
    return data2

# 사용자 최고 레벨
def level_max(train_combat,data):
    level_max = train_combat[['acc_id','level']]
    level_max = level_max.groupby('acc_id').max()
    level_max = level_max.reset_index()
    level_max.rename(columns={'level':'level_max'},inplace=True)
    data2 = pd.merge(data, level_max, on = 'acc_id')
    return data2

# 클래스를 무과금 가능 클래스(저렙과 엮으면 시너지?) vS 무과금 힘든 클래스, pk 유리 클래스 vs pk 약한 클래스

# 접속일자 단순 합(범위: 1~406)
def day_sum(train_combat, data):
    day_sum = train_combat[['day','acc_id','char_id']]
    day_sum = day_sum.groupby(['day','acc_id']).sum().reset_index()
    day_sum = day_sum.groupby(['acc_id']).sum().reset_index()
    day_sum = day_sum.drop('char_id',axis=1) 
    day_sum.rename(columns={'day':'day_sum'},inplace=True)

    data2 = pd.merge(data, day_sum, on='acc_id')
    return data2
# 각 클래스별로 사용 횟수를 다 더해서 비교하면 의미가 있을까??
# 날짜가 들어가는 건 의미가 없을 것 같고, 전체 캐릭터 중에서 어떤 클래스를 복수로 운영하는지를 보는 게 좋을 것 같다.
def class_count(train_combat, data):
    class_dum =pd.get_dummies(train_combat['class'], prefix="C")
    combat_dum = pd.concat([train_combat[['day','acc_id','char_id']], class_dum], axis=1)
    combat_dum = combat_dum.groupby(['day','acc_id']).sum().reset_index().drop('char_id',axis=1)
    combat_dum = combat_dum.groupby(['acc_id']).max().reset_index().drop('day',axis=1)
    
    data2 = pd.merge(data, combat_dum, on='acc_id')
    return data2

In [148]:
# 모든 변수 추가
data = num_combat(train_combat, data1)
data = std_combat(train_combat, data)
data = acc_std_combat(train_combat, data)

data = deff_level(train_combat, data)
data = level_max(train_combat, data)
data = char_day_count(train_combat, data)
data = char_max(train_combat, data)
data = day_sum(train_combat, data)
data = class_count(train_combat, data)

# day_mean : day_sum / 접속일수
data['day_mean'] = data['day_sum'] / data['day_count']

# 장사꾼 유저
data['merchant'] = np.where((data1['npc_kill']==0) & (data1['solo_exp']==0) & (data1['party_exp']==0) &
                                         (data1['rich_monster']==0) & (data1['death']==0) & (data1['fishing']==0) &
                                         (data1['pledge_cnt']==0) & (data1['random_attacker_cnt']==0) & (data1['random_defender_cnt']==0) & 
                                         (data1['temp_cnt']==0) & (data1['same_pledge_cnt']==0) & (data1['etc_cnt']==0) & 
                                         (data1['private_shop']>0), 1, 0)

C:\Users\SAMSUNG\Desktop\new\anacanda\envs\mypython\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\SAMSUNG\Desktop\new\anacanda\envs\mypython\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\SAMSUNG\Desktop\new\anacanda\envs\mypython\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on 

In [149]:
data.columns

Index(['acc_id', 'playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp',
       'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'private_shop', 'game_money_change', 'enchant_count', 'pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'day_pledge',
       'day_attack', 'day_defend', 'day_temp', 'day_same', 'day_etc',
       'std_pledge', 'std_attack', 'std_defend', 'std_temp', 'std_same',
       'std_etc', 'acc_std_pledge', 'acc_std_attack', 'acc_std_defend',
       'acc_std_temp', 'acc_std_same', 'acc_std_etc', 'deff_level',
       'level_max', 'day_count', 'char_count', 'char_max', 'day_sum', 'C_0',
       'C_1', 'C_2', 'C_3', 'C_4', 'C_5', 'C_6', 'C_7', 'day_mean',
       'merchant'],
      dtype='object')

In [150]:
# act_40000_best랑 중복되거나 쓸데 없는 컬럼 삭제
# merge 시 'Unnamed: 0'가 생겼다면 drop할 columns 목록에 추가 해주기
data2 = data.drop(['playtime', 'npc_kill', 'solo_exp', 'party_exp',
       'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery',
       'fishing', 'private_shop', 'game_money_change', 'enchant_count',
       'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent'],axis=1)

In [151]:
act_40000_best = pd.merge(act_40000_best, data2, on = 'acc_id').drop(['char_id','day'],axis=1)

# ---------------------payment------------------------

In [152]:
# 결제 데이터 추가
payment = train_payment.groupby(['acc_id']).sum().reset_index()
act_40000_best = pd.merge(act_40000_best, payment, on ='acc_id', how='outer').fillna(0)

# ---------------------trade------------------------

In [153]:
# 일별 거래 횟수 추가
# 주는 유저
train_trade_source = train_trade[['day', 'time', 'type', 'server', 'source_acc_id', 
                            'source_char_id', 'item_type', 'item_amount','item_price']]
# 받는 유저
train_trade_target = train_trade[['day', 'time', 'type', 'server', 'target_acc_id', 
                            'target_char_id', 'item_type', 'item_amount','item_price']]

train_trade_source = train_trade_source.rename(columns = {'source_acc_id': 'acc_id', 'source_char_id':'char_id',
                                                         'item_amount': 'item_amount_s', 'item_price': 'item_price_s'})
train_trade_target = train_trade_target.rename(columns = {'target_acc_id': 'acc_id', 'target_char_id':'char_id',
                                                         'item_amount': 'item_amount_t', 'item_price': 'item_price_t'})

# 거래가 있는 날은 모두 1 (일별 거래횟수를 count하기 위해 추가)
train_trade_source['num_trade_s'] = 1
train_trade_target['num_trade_t'] = 1

# '시간' 컬럼을 그룹별로 합침(time, item_type, server 제거됨  )
train_trade_source = train_trade_source.groupby(['day','acc_id','char_id']).sum()
train_trade_source = train_trade_source.reset_index()

train_trade_target = train_trade_target.groupby(['day','acc_id','char_id']).sum()
train_trade_target = train_trade_target.reset_index()

# 받는 유저의 item price는 '-'로 변경 (사는 거니까)
train_trade_target['item_price_t'] = train_trade_target['item_price_t']*(-1)

# playtime을 기준으로 NaN이면 40000명 목록에 없는 유저로 판단하여 제거
activity = train_activity.groupby('acc_id').sum().reset_index()
standard = activity[['acc_id','day','playtime']]
source = pd.merge(standard, train_trade_source , on=['day','acc_id'], how = 'outer').dropna(subset = ['playtime']).drop(['char_id','playtime'],axis=1)
target = pd.merge(standard, train_trade_target , on=['day','acc_id'], how = 'outer').dropna(subset = ['playtime']).drop(['char_id','playtime'],axis=1)

trade = pd.merge(source, target , on=['day','acc_id'], how = 'outer')
trade = trade.groupby(['acc_id','day']).sum().reset_index()

# 일별 거래 횟수 추가
num_trade = trade[['acc_id','num_trade_s','num_trade_t']]

In [162]:
act_40000_best = pd.merge(act_40000_best, trade[['acc_id', 'item_amount_s', 'item_price_s',
       'num_trade_s', 'item_amount_t', 'item_price_t',
       'num_trade_t']], on ='acc_id').fillna(0)

In [205]:
act_40000_best.shape

(40000, 118)

### 변수 추가 전 activity에 forgive = death - revive 만 추가한 후 flatten

- 보통 revive가 조금 높은데 중간중간 death가 높은 캐릭터들이 있다.
- User가 흥미를 잃을 수 있는 척도로 예측

In [287]:
tr_activity = train_activity.copy()

In [288]:
tr_activity['forgive'] = tr_activity['death'] - tr_activity['revive']

In [291]:
df = tr_activity[[col for col in tr_activity.columns if col not in ['server', 
                                                                          'char_id']]
                  ].groupby(['day', 'acc_id']).sum().reset_index()
df_grouped = df.groupby('day')
p = df_grouped.get_group(1)
for i in range(2, 29):
    p = pd.merge(p, df_grouped.get_group(i), on='acc_id', how='outer',
                 suffixes=('_'+str(i-1), '_'+str(i)))
p = p[[col for col in p.columns if 'day' not in col]]
p = p.fillna(0).set_index('acc_id')
df = p.reset_index()
act_40000_best = pd.merge(act_40000_best, df, on = 'acc_id')

# ---------------------pledge------------------------

In [165]:
# dir = 'C:/Users/SAMSUNG/Desktop/new/빅콘테스트/2019빅콘테스트_챔피언스리그_데이터_수정/train_pledge.csv'

def preprocessing_pledge(pledge_dir):
    # file load
    tr_pledge = pd.read_csv(pledge_dir)
    
    
    # 공지에 올라온 error 부분 제거
    del tr_pledge['non_combat_play_time']
    
    # 가입한 혈맹의 순위
    bbb = tr_pledge.pledge_id.value_counts().to_frame().reset_index()
    bbb.columns = ['pledge_id','count']
    bbb['rank'] = bbb['count'].rank(ascending = False, method = 'min')
    pledge_rank = {}
    # dictionary를 활용해 원데이터의 pledge_id에 rank값을 mapping
    for i, j in enumerate(list(bbb['rank'])):
        pledge_rank[bbb['pledge_id'][i]] = j
    tr_pledge['pledge_rank'] = tr_pledge['pledge_id'].map(pledge_rank)


    # 혈맹원의 합
    pledge_member_num = {}
    for i in tr_pledge.pledge_id:
        if i not in pledge_member_num.keys():
            pledge_member_num[i] = 0
        pledge_member_num[i] += 1
    tr_pledge['pledge_member_num'] = tr_pledge['pledge_id'].map(pledge_member_num)


    # acc_id 기준으로 데이터 압축
    group = tr_pledge.groupby(['acc_id', 'day']).sum().reset_index()
    groups = group.groupby(['acc_id']).sum().reset_index()


    # 접속일 변수 log_in_freq 생성
    freq = []
    for i in group.acc_id.unique():
        freq.append([i,group[group.acc_id == i].shape[0]])
    new = pd.DataFrame(sorted(freq))
    new.columns = ['acc_id', 'log_in_freq']
    merge_df = pd.merge(groups, new, how = 'left', on = 'acc_id')


    # 유저별 가입한 혈맹 수
    act_pledge_num = {}
    for i in tr_pledge.acc_id.unique():
        act_pledge_num[i] = tr_pledge[tr_pledge.acc_id == i].pledge_id.nunique()
    merge_df['join_pledge_num'] = merge_df['acc_id'].map(act_pledge_num)

    
    # 유저별 a서버, b서버 접속 횟수
    acc_id = []
    a_server_num = []
    b_server_num = []

    for i in tr_pledge.acc_id.unique():
        a_count = 0
        b_count = 0
        li = list(tr_pledge[tr_pledge.acc_id == i].server)
        for j in li:
            if j[0] == 'a':
                a_count += 1
            else:
                b_count += 1
        acc_id.append(i)
        a_server_num.append(a_count)
        b_server_num.append(b_count)
    
    df = pd.DataFrame({'acc_id' : acc_id,
                  'a_server_num' : a_server_num,
                  'b_server_num' : b_server_num})
    merge_df = pd.merge(merge_df, df, on = 'acc_id')
    

    # 일자별 혈맹 활동 내역 flatten (pledge_rank는 flatten을 안시키는게 퍼포먼스 향상에 더 좋음)
    df = tr_pledge[[col for col in tr_pledge.columns if col not in ['server', 'char_id','pledge_id',
                                                                   'pledge_rank']]
                  ].groupby(['day', 'acc_id']).sum().reset_index()
    df_grouped = df.groupby('day')
    p = df_grouped.get_group(1)
    for i in range(2, 29):
        p = pd.merge(p, df_grouped.get_group(i), on='acc_id', how='outer',
                     suffixes=('_'+str(i-1), '_'+str(i)))
    p = p[[col for col in p.columns if ('day' not in col) & ('combat_char_cnt' not in col)]]
    p = p.fillna(0).set_index('acc_id')
    df = p.reset_index()
    merge_flatten_df = pd.merge(merge_df, df, on = 'acc_id')
    
    return merge_flatten_df

In [166]:
pledge_dir = 'C:/Users/SAMSUNG/Desktop/new/빅콘테스트/2019빅콘테스트_챔피언스리그_데이터_수정/train_pledge.csv'

merge_pledge = preprocessing_pledge(pledge_dir)

In [296]:
# pledge에도 day가 있으나 변수로써 유용하기에 drop하지 않음

merge_all = pd.merge(act_40000_best, merge_pledge.drop(['char_id', 'pledge_id'], axis = 1),
                    on = 'acc_id', how='outer').fillna(0)

In [297]:
merge_all.shape

(40000, 778)

# -------------------------------------merge all end------------------------------------------

# merge all을 csv로 파일 저장

In [ ]:
merge_all.to_csv('./merge_all.csv', index = False)

## -----------------------------------------중간중간 실험해보는 영역---------------------------------------------

In [214]:
use_col = ['pledge_id', 'day', 'acc_id', 'char_id', 'server']
test2 = train_pledge[use_col].sort_values(use_col)

main_server_ind, bat_server_ind = [], []
prepare_ind = test2.groupby(['pledge_id', 'day', 'acc_id', 'char_id']).count().values.ravel()
for i in prepare_ind:
    if i > 1:
        for j in range(i):
            main_server_ind.append(False)
            bat_server_ind.append(True)
    else:
        main_server_ind.append(True)
        bat_server_ind.append(False)

In [215]:

train_main_pledge = train_pledge.sort_values(use_col)[main_server_ind]
train_bat_pledge = train_pledge.sort_values(use_col)[bat_server_ind]

In [228]:
pledge_id_stats = train_main_pledge.groupby(['acc_id', 'char_id', 'day']).agg({'pledge_id' : ['first', 'last', 'count']})

### activity flatten 전후 비교실험

- 결과적으로 flatten 시키는 것이 accuracy를 증가시키고 rmse를 줄인다.

In [234]:
train_activity.columns

Index(['day', 'acc_id', 'char_id', 'server', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count'],
      dtype='object')

In [236]:
train_activity_non_flatten = train_activity.groupby('acc_id').sum().reset_index()

In [238]:
df = train_activity[[col for col in train_activity.columns if col not in ['server', 
                                                                          'char_id']]
                  ].groupby(['day', 'acc_id']).sum().reset_index()
df_grouped = df.groupby('day')
p = df_grouped.get_group(1)
for i in range(2, 29):
    p = pd.merge(p, df_grouped.get_group(i), on='acc_id', how='outer',
                 suffixes=('_'+str(i-1), '_'+str(i)))
p = p[[col for col in p.columns if 'day' not in col]]
p = p.fillna(0).set_index('acc_id')
df = p.reset_index()
train_activity_flatten = pd.merge(train_activity_non_flatten, df, on = 'acc_id')

In [247]:
# label 합치기 
train_activity_flatten = pd.merge(train_activity_flatten, train_label, on='acc_id')
train_activity_non_flatten = pd.merge(train_activity_non_flatten, train_label, on='acc_id')

In [248]:
train_activity_flatten.shape, train_activity_non_flatten.shape

((40000, 384), (40000, 20))

In [249]:
train_activity_non_flatten.columns

Index(['acc_id', 'day', 'char_id', 'playtime', 'npc_kill', 'solo_exp',
       'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'survival_time', 'amount_spent', 'secession',
       'total_spent'],
      dtype='object')

In [277]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_activity_non_flatten.drop(['acc_id',
                                                                           'char_id',
                                                                           'amount_spent',
                                                                           'total_spent',
                                                                          'secession',
                                                                          'survival_time'], axis=1),
                                                    train_activity_non_flatten.total_spent,
                                                    test_size = 0.3, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25, random_state=42)

X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape

((21000, 14), (7000, 14), (12000, 14), (21000,), (7000,), (12000,))

In [280]:
import xgboost as xgb

params = {'n_estimators' : 475,
         'learning_rate' : 0.1,
          'nthread' : 15,
          'num_boost_round' : 300,
         'max_depth' : 6,
          'eval_metric' : 'auc',
          'gamma' : 1,
          'subsample' : 0.9}

VALID = True
np.random.seed(42)
if VALID == True:
    
    tr_data = xgb.DMatrix(X_train, y_train)
    va_data = xgb.DMatrix(X_valid, y_valid)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model = xgb.train(params, tr_data, 500, watchlist, verbose_eval = 50, early_stopping_rounds=30)

C:\Users\SAMSUNG\Desktop\new\anacanda\envs\mypython\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBoostError: [03:03:34] D:\Build\xgboost\xgboost-0.90.git\src\metric\rank_metric.cc:200: Check failed: !auc_error: AUC: the dataset only contains pos or neg samples

In [260]:
# flatten
from sklearn.metrics import accuracy_score, f1_score
y_pred = np.where(model.predict(xgb.DMatrix(X_test)) > 0.5, 1, 0)
accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)

(0.7594166666666666, 0.7264803410705827)

In [263]:
# non_flatten
from sklearn.metrics import accuracy_score, f1_score
y_pred = np.where(model.predict(xgb.DMatrix(X_test)) > 0.5, 1, 0)
accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)

(0.7394166666666667, 0.7045261268071434)

In [278]:
import xgboost as xgb

params = {'n_estimators' : 475,
         'learning_rate' : 0.1,
          'nthread' : 15,
          'num_boost_round' : 300,
         'max_depth' : 6,
          'objective': 'reg:linear',
          'booster' : 'gblinear',
          'gamma' : 0,
          'subsample' : 0.9}

VALID = True
np.random.seed(42)
if VALID == True:
    
    tr_data = xgb.DMatrix(X_train, y_train)
    va_data = xgb.DMatrix(X_valid, y_valid)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model = xgb.train(params, tr_data, 10000, watchlist, verbose_eval = 50, early_stopping_rounds=30)

[03:02:07] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:6.89451	valid-rmse:6.76606
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 30 rounds.


C:\Users\SAMSUNG\Desktop\new\anacanda\envs\mypython\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[50]	train-rmse:6.20555	valid-rmse:6.05879
[100]	train-rmse:6.19183	valid-rmse:6.04654
[150]	train-rmse:6.18632	valid-rmse:6.04211
[200]	train-rmse:6.18395	valid-rmse:6.0404
[250]	train-rmse:6.18284	valid-rmse:6.03965
[300]	train-rmse:6.18224	valid-rmse:6.03931
[350]	train-rmse:6.18187	valid-rmse:6.03905
[400]	train-rmse:6.18158	valid-rmse:6.03884
[450]	train-rmse:6.18135	valid-rmse:6.03864
[500]	train-rmse:6.18113	valid-rmse:6.03842
[550]	train-rmse:6.18093	valid-rmse:6.03821
[600]	train-rmse:6.18075	valid-rmse:6.038
[650]	train-rmse:6.18058	valid-rmse:6.03779
[700]	train-rmse:6.18041	valid-rmse:6.03759
[750]	train-rmse:6.18025	valid-rmse:6.03739
[800]	train-rmse:6.18011	valid-rmse:6.03721
[850]	train-rmse:6.17997	valid-rmse:6.03703
[900]	train-rmse:6.17984	valid-rmse:6.03686
[950]	train-rmse:6.17971	valid-rmse:6.03669
[1000]	train-rmse:6.17959	valid-rmse:6.03655
[1050]	train-rmse:6.17948	valid-rmse:6.03639
[1100]	train-rmse:6.17937	valid-rmse:6.03624
[1150]	train-rmse:6.17927	valid-r

In [272]:
# flatten
from sklearn.metrics import mean_squared_error

y_pred = model.predict(xgb.DMatrix(pd.DataFrame(X_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 5.985313


In [279]:
# non_flatten
from sklearn.metrics import mean_squared_error

y_pred = model.predict(xgb.DMatrix(pd.DataFrame(X_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 6.088201


#### 단순히 forgive = death - revive 생성했을 때 label과 상관관계 보기

In [283]:
aaa = train_activity_non_flatten.copy()

In [284]:
aaa['forgive'] = aaa['death'] - aaa['revive']

In [285]:
aaa.corr()

,acc_id,day,char_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,...,exp_recovery,fishing,private_shop,game_money_change,enchant_count,survival_time,amount_spent,secession,total_spent,forgive
acc_id,1.000000,0.005646,0.003514,-0.002883,0.003232,0.006622,0.000038,-0.000440,0.006187,0.005465,...,0.004056,0.004371,-0.003704,0.007020,0.003881,-0.001575,0.005493,0.002341,0.005891,-0.005169
day,0.005646,1.000000,0.835491,0.139839,0.216997,0.179426,0.032984,0.200625,0.467523,0.142915,...,0.064896,0.188757,-0.071009,0.013625,0.025029,0.246270,-0.025333,-0.195488,0.177294,-0.003904
char_id,0.003514,0.835491,1.000000,0.142523,0.208111,0.140512,0.018001,0.139587,0.408703,0.126555,...,0.052835,0.171869,-0.052737,-0.005482,0.023481,0.214848,-0.021271,-0.173356,0.154327,-0.002609
playtime,-0.002883,0.139839,0.142523,1.000000,0.301763,-0.178753,0.047175,-0.163277,-0.065355,0.099679,...,-0.061564,0.180046,0.530384,-0.030948,-0.001402,0.272280,-0.073376,-0.263874,-0.012882,0.258804
npc_kill,0.003232,0.216997,0.208111,0.301763,1.000000,0.039558,-0.026438,-0.057688,0.257203,0.157437,...,0.066279,0.170798,-0.329173,-0.001698,0.029842,0.161885,-0.010778,-0.128968,0.210841,0.201987
solo_exp,0.006622,0.179426,0.140512,-0.178753,0.039558,1.000000,0.111297,0.596230,0.225991,0.124409,...,0.110392,-0.034974,-0.189649,0.017481,0.009536,-0.097808,-0.002585,0.161375,0.013815,-0.005327
party_exp,0.000038,0.032984,0.018001,0.047175,-0.026438,0.111297,1.000000,0.131059,-0.015924,0.145128,...,-0.014778,-0.088073,-0.090271,0.005840,-0.012180,-0.102179,-0.025621,0.117304,-0.076787,0.109724
quest_exp,-0.000440,0.200625,0.139587,-0.163277,-0.057688,0.596230,0.131059,1.000000,0.146717,0.061202,...,0.087900,-0.026317,-0.123870,0.017681,0.013422,-0.072927,0.004839,0.109560,0.029925,-0.039363
rich_monster,0.006187,0.467523,0.408703,-0.065355,0.257203,0.225991,-0.015924,0.146717,1.000000,0.134245,...,0.147056,0.317342,-0.260915,0.002156,0.041531,0.189934,0.011196,-0.153172,0.262959,-0.090469
death,0.005465,0.142915,0.126555,0.099679,0.157437,0.124409,0.145128,0.061202,0.134245,1.000000,...,0.024690,0.012570,-0.121894,0.002468,0.004781,-0.004280,-0.005295,0.025378,0.069743,0.168167
